<a href="https://colab.research.google.com/github/minjeong-kim-git/GNUVILL_AI/blob/main/GNUVILL_%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5_%EC%8A%A4%ED%84%B0%EB%94%94(%EC%A4%91%EA%B8%89)_6%EC%A3%BC%EC%B0%A8_%EC%8B%A4%EC%8A%B5_%EB%8B%B5%EC%95%88_%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GNUVILL 인공지능 스터디(중급) 6주차 실습: 로지스틱 회귀

유방암 데이터 분석

---

##1. 라이브러리

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

## 2. 데이터 준비하기

In [ ]:
b_cancer = load_breast_cancer()
print(b_cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [ ]:
b_cancer_df = pd.DataFrame(b_cancer.data, columns =b_cancer.feature_names)
b_cancer_df['diagnosis'] = b_cancer.target
print('데이터셋 크기: ', b_cancer_df.shape)
b_cancer_df.head()

데이터셋 크기:  (569, 31)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0




*   **.info()**:데이터의 통계 정보 확인





In [ ]:
b_cancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

## 3. 데이터 전처리

독립변수로 사용할 데이터를 평균이 0, 분산이 1이되는 정규분포 형태로 수정

스케일링을 통해 다차원의 값들을 비교 분석하기 쉽게 만들어주며, 자료의 오버플로우(overflow)나 언더플로우(underflow)를 방지 하고, 독립 변수의 공분산 행렬의 조건수(condition number)를 감소시켜 최적화 과정에서의 안정성 및 수렴 속도를 향상 시킨다.

*   **sklearn.preprocessing**: 데이터 스케일링, 변환을 담당하는 패키지
*   **StandardScaler**
  :평균을 제거하고 데이터를 단위 분산으로 조정.
  
그러나 StandardScaler의 경우 이상치가 있다면 평균과 표준편차에 영향을 미쳐 변환된 데이터의 확산은 매우 달라지게 된다.

**따라서 이상치가 있는 경우 균형 잡힌 척도를 보장할 수 없다.**



*   **scaler.fit_transform**: scaler의 단축 메서드로 학습용 데이터에 적용.
  
  fit으로 데이터 변환을 학습하고, transform으로 실제 데이터의 스케일을 조정





In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
b_cancer_scaled = scaler.fit_transform(b_cancer.data)

In [ ]:
print(b_cancer_scaled[0])

[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
  2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
  2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
  1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
  1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]


In [ ]:
print(b_cancer_scaled[29])

[ 0.97777802 -0.98659467  0.94865013  0.85383059  0.15013907  0.21527015
  0.12493055  0.78957583 -0.2651265  -0.1853673   0.70425773 -0.71549273
  0.88558044  0.45681976 -0.47134541  0.27116829  0.07215909  0.28287142
 -0.1564696  -0.02001096  0.77465644 -1.00266557  0.82324445  0.60897062
 -0.30109094  0.17134395 -0.11172705  0.47192976 -0.23418298 -0.2635475 ]


## 4. 로지스틱 회귀 모형 구축

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
# X, Y 설정
Y = b_cancer_df['diagnosis']
X = b_cancer_scaled

In [ ]:
# 훈련용, 테스트용 데이터 분리
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [ ]:
lr_b_cancer = LogisticRegression() # 로지스틱 회귀 모형 생성
lr_b_cancer.fit(X_train, Y_train) # 로지스틱 회귀 모형 훈련

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
Y_predict = lr_b_cancer.predict(X_test)

In [ ]:
lr_b_cancer.coef_

array([[-0.25965337, -0.58891309, -0.27527119, -0.35070364, -0.13501664,
         0.41458176, -0.67094946, -0.74096029, -0.37987661,  0.03289321,
        -1.35087219,  0.14092099, -0.90852156, -0.98001244,  0.25957915,
         0.92374666, -0.13448413, -0.34937963,  0.16812679,  0.91359164,
        -0.8473647 , -0.91043792, -0.73517119, -0.84764528, -0.56888501,
         0.17670599, -0.82594672, -1.08228373, -0.48409262, -0.60758107]])

In [ ]:
lr_b_cancer.intercept_

array([0.04258607])

## 5. 로지스틱 회귀 모형 성능 확인

  https://gaussian37.github.io/ml-concept-ml-evaluation/#precision-1

*   **confusion_matrix**: 타겟의 원래 클래스와 모형이 예측한 클래스가 일치하는지는 갯수로 센 결과를 표나 나타냄
*   **accuracy_score**: 전체 대비 정확하게 예측한 개수의 비율
*   **precision_score**: Positive라고 예측한 것 중에서 얼마나 잘 예측했는지에 대한 비율
*   **recall_score**: 실제 Positive한 것 중에서 얼마나 잘 예측하였는지에 대한 비율
*   **f1_score**: 두 지표를 평균값을 통해 하나의 값으로 나타내는 방법







In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score



*   **오차행렬(confusion matrix)**: 학습된 분류 모델이 예측을 수행하면서 얼마나 헷갈리고(confused) 있는지도 함께 보여주는 지표로 이진 분류의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측 오류가 발생하고 있는지를 함께 나타냄.



In [ ]:
# 오차행렬 확인
confusion_matrix(Y_test, Y_predict)

array([[ 60,   3],
       [  1, 107]])

In [ ]:
accuracy = accuracy_score(Y_test, Y_predict)
precision = precision_score(Y_test, Y_predict)
recall = recall_score(Y_test, Y_predict)
f1 = f1_score(Y_test, Y_predict)

In [ ]:
print('정확도: {0:.3f}, 정밀도: {1:.3f}, 재현율: {2:.3f}, F1: {3:.3f}'.format(accuracy, precision, recall, f1))

정확도: 0.977, 정밀도: 0.973, 재현율: 0.991, F1: 0.982


In [ ]:
lr_b_cancer.predict([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

array([1])